In [3]:
import ast

In [4]:
def calculate_sum_of_three_elements(a, b, c):
    return a + b + c

def calculate_sum_of_three_elements2(a: int, b: int, c: int):
    a += b
    a += c
    return a

def calculate_sum_of_two_elements(a: int, b: int) -> int:
    return a + b

def calculate_sum_of_three_elements3(a: int, b: int, c: int) -> int:
    return calculate_sum_of_two_elements(calculate_sum_of_three_elements(a, b), c)

def calculate_sum_of_three_elements4(a, b, c):
    return calculate_sum_of_two_elements(a, c) + b

def calculate_sum_of_three_elements5(a, b, c):
    return sum([a, b, c])

In [5]:
csotes = """
def calculate_sum_of_three_elements(a, b, c):
    return a + b + c

def calculate_sum_of_three_elements2(a: int, b: int, c: int):
    a += b
    a += c
    return a

def calculate_sum_of_two_elements(a: int, b: int) -> int:
    return a + b

def calculate_sum_of_three_elements3(a: int, b: int, c: int) -> int:
    return calculate_sum_of_two_elements(calculate_sum_of_three_elements(a, b), c)

def calculate_sum_of_three_elements4(a, b, c):
    return calculate_sum_of_two_elements(a, c) + b

def calculate_sum_of_three_elements5(a, b, c):
    return sum([a, b, c])
""".strip().split("\n\n")

In [6]:
csotes

['def calculate_sum_of_three_elements(a, b, c):\n    return a + b + c',
 'def calculate_sum_of_three_elements2(a: int, b: int, c: int):\n    a += b\n    a += c\n    return a',
 'def calculate_sum_of_two_elements(a: int, b: int) -> int:\n    return a + b',
 'def calculate_sum_of_three_elements3(a: int, b: int, c: int) -> int:\n    return calculate_sum_of_two_elements(calculate_sum_of_three_elements(a, b), c)',
 'def calculate_sum_of_three_elements4(a, b, c):\n    return calculate_sum_of_two_elements(a, c) + b',
 'def calculate_sum_of_three_elements5(a, b, c):\n    return sum([a, b, c])']

In [7]:
trees = [ast.parse(csote) for csote in csotes]

In [8]:
for tree in trees:
    print(ast.dump(tree, indent=4))
    print("\n\n")

Module(
    body=[
        FunctionDef(
            name='calculate_sum_of_three_elements',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b'),
                    arg(arg='c')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=BinOp(
                            left=Name(id='a', ctx=Load()),
                            op=Add(),
                            right=Name(id='b', ctx=Load())),
                        op=Add(),
                        right=Name(id='c', ctx=Load())))],
            decorator_list=[])],
    type_ignores=[])



Module(
    body=[
        FunctionDef(
            name='calculate_sum_of_three_elements2',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(
        

In [9]:
def ast_to_dict(node):
    """Convert an AST node to a dictionary."""
    if isinstance(node, ast.AST):
        fields = {}
        for name, value in ast.iter_fields(node):
            fields[name] = ast_to_dict(value)
        return {
            "type": type(node).__name__,
            "fields": fields,
        }
    elif isinstance(node, list):
        return [ast_to_dict(x) for x in node]
    else:
        return node

In [10]:
ast_to_dict(trees[0])

{'type': 'Module',
 'fields': {'body': [{'type': 'FunctionDef',
    'fields': {'name': 'calculate_sum_of_three_elements',
     'args': {'type': 'arguments',
      'fields': {'posonlyargs': [],
       'args': [{'type': 'arg',
         'fields': {'arg': 'a', 'annotation': None, 'type_comment': None}},
        {'type': 'arg',
         'fields': {'arg': 'b', 'annotation': None, 'type_comment': None}},
        {'type': 'arg',
         'fields': {'arg': 'c', 'annotation': None, 'type_comment': None}}],
       'vararg': None,
       'kwonlyargs': [],
       'kw_defaults': [],
       'kwarg': None,
       'defaults': []}},
     'body': [{'type': 'Return',
       'fields': {'value': {'type': 'BinOp',
         'fields': {'left': {'type': 'BinOp',
           'fields': {'left': {'type': 'Name',
             'fields': {'id': 'a', 'ctx': {'type': 'Load', 'fields': {}}}},
            'op': {'type': 'Add', 'fields': {}},
            'right': {'type': 'Name',
             'fields': {'id': 'b', 'ctx': {

In [11]:
class RenameVars(ast.NodeTransformer):
    def __init__(self):
        self.arg_num = 1
        self.var_num = 1
        self.var_map = {}
        
    def get_new_arg_name(self):
        name = f"arg{self.arg_num}"
        self.arg_num += 1
        return name
    
    def get_new_var_name(self):
        name = f"var{self.var_num}"
        self.var_num += 1
        return name
    
    def visit_FunctionDef(self, node):
        # Rename function arguments
        for arg in node.args.args:
            arg.arg = self.get_new_arg_name()

        # Rename variables in function body
        for stmt in node.body:
            if isinstance(stmt, ast.Assign):
                for target in stmt.targets:
                    if isinstance(target, ast.Name):
                        target.id = self.get_new_var_name()
                        
        return node

    def visit_Name(self, node):
        # Rename variables
        if isinstance(node.ctx, ast.Load):
            old_name = node.id
            if old_name in self.var_map:
                node.id = self.var_map[old_name]
            else:
                new_name = self.get_new_var_name()
                self.var_map[old_name] = new_name
                node.id = new_name
        return node

def rename_vars(code):
    ast_tree = ast.parse(code)
    renamer = RenameVars()
    renamed_ast = renamer.visit(ast_tree)
#     return ast.unparse(renamed_ast)
    return ast.dump(renamed_ast, indent=4)

In [12]:
code = """
def foo(x):
    y = 1 + x
    z = foo(y)
    return z
"""

print(rename_vars(code))
print(ast.dump(ast.parse(code), indent=4))

Module(
    body=[
        FunctionDef(
            name='foo',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arg1')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='var1', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=1),
                        op=Add(),
                        right=Name(id='x', ctx=Load()))),
                Assign(
                    targets=[
                        Name(id='var2', ctx=Store())],
                    value=Call(
                        func=Name(id='foo', ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[])),
                Return(
                    value=Name(id='z', ctx=Load()))],
            decorato

In [13]:
code = """
def foo(x):
    y = 1 + x
    z = foo(y)
    return z
"""
ast_tree = ast.parse(code)
renamer = RenameVars()
renamed_ast = renamer.visit(ast_tree)

In [14]:
print(ast.dump(renamed_ast, indent=4))

Module(
    body=[
        FunctionDef(
            name='foo',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arg1')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='var1', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=1),
                        op=Add(),
                        right=Name(id='x', ctx=Load()))),
                Assign(
                    targets=[
                        Name(id='var2', ctx=Store())],
                    value=Call(
                        func=Name(id='foo', ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[])),
                Return(
                    value=Name(id='z', ctx=Load()))],
            decorato

In [15]:
import astor

print(astor.to_source(renamed_ast))

def foo(arg1):
    var1 = 1 + x
    var2 = foo(y)
    return z



In [16]:
import ast

# Example code to normalize
code = '''
def foo(x: int, y: int) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y
'''

# Parse the code into an AST
tree = ast.parse(code)

# Define a function to generate new variable names
var_counter = 1
def new_var():
    global var_counter
    name = f'var{var_counter}'
    var_counter += 1
    return name

# Define a function to rename variables
def rename_variables(node):
    global var_counter
    if isinstance(node, ast.Name):
        node.id = new_var()
    elif isinstance(node, ast.arg):
        node.arg = f'arg{var_counter}'
        var_counter += 1
    for child in ast.iter_child_nodes(node):
        rename_variables(child)

# Define a function to remove function typings
def remove_typings(node):
    if isinstance(node, ast.FunctionDef):
        node.returns = None
        for arg in node.args.args:
            arg.annotation = None
    for child in ast.iter_child_nodes(node):
        remove_typings(child)

# Rename variables and remove function typings
rename_variables(tree)
remove_typings(tree)

# Define a function to replace variables with arguments if they are the same
def replace_variables_with_arguments(node):
    if isinstance(node, ast.Name):
        if node.id.startswith('var') and node.id[3:].isdigit():
            index = int(node.id[3:]) - 1
            if isinstance(node.parent, ast.FunctionDef):
                if index < len(node.parent.args.args):
                    arg = node.parent.args.args[index]
                    node.id = arg.arg
            elif isinstance(node.parent, ast.Assign):
                targets = node.parent.targets
                if index < len(targets):
                    target = targets[index]
                    if isinstance(target, ast.Name):
                        node.id = target.id
    for child in ast.iter_child_nodes(node):
        child.parent = node
        replace_variables_with_arguments(child)

# Replace variables with arguments if they are the same
replace_variables_with_arguments(tree)

# Normalize the AST
ast.fix_missing_locations(tree)

# Print the normalized AST
print(ast.unparse(tree))

def foo(arg1, arg3):
    var5 = var6 + var7
    if var8 > 10:
        return var9
    else:
        return var10 * var11


In [17]:
import ast
import inspect

def foo(x: int, y: int) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y

def normalize_function(code):
    # Parse the code into an AST
    tree = ast.parse(code)

    # Rename variables and arguments
    def rename_variables_and_arguments(node):
        if isinstance(node, ast.Name):
            if node.id.startswith('arg') and node.id[3:].isdigit():
                index = int(node.id[3:]) - 1
                node.id = f'arg{index + 1}'
            elif node.id.startswith('var') and node.id[3:].isdigit():
                index = int(node.id[3:]) - 1
                node.id = f'var{index + 1}'
        for child in ast.iter_child_nodes(node):
            child.parent = node
            rename_variables_and_arguments(child)

    rename_variables_and_arguments(tree)

    # Remove function typings
    def remove_function_typings(node):
        if isinstance(node, ast.AnnAssign):
            node.annotation = None
        elif isinstance(node, ast.arg):
            node.annotation = None
        for child in ast.iter_child_nodes(node):
            child.parent = node
            remove_function_typings(child)

    remove_function_typings(tree)

    # Replace variables with arguments if they are the same
    def replace_variables_with_arguments(node):
        if isinstance(node, ast.Name):
            if node.id.startswith('var') and node.id[3:].isdigit():
                index = int(node.id[3:]) - 1
                if isinstance(node.parent, ast.FunctionDef):
                    if index < len(node.parent.args.args):
                        arg = node.parent.args.args[index]
                        node.id = arg.arg
                elif isinstance(node.parent, ast.Assign):
                    targets = node.parent.targets
                    if index < len(targets):
                        target = targets[index]
                        if isinstance(target, ast.Name):
                            node.id = target.id
        for child in ast.iter_child_nodes(node):
            child.parent = node
            replace_variables_with_arguments(child)

    replace_variables_with_arguments(tree)

    # Generate the new code
    new_code = ast.unparse(tree)

    return new_code

# Example usage
code = inspect.getsource(foo)
new_code = normalize_function(code)
print(new_code)

def foo(x, y) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y


In [18]:
def foo(x: int, y: int) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y

In [19]:
import inspect

In [20]:
inspect.getsource(foo)

'def foo(x: int, y: int) -> int:\n    z = x + y\n    if z > 10:\n        return z\n    else:\n        return x * y\n'

In [21]:
import ast
import inspect

def simplify(func):
    # Parse the function code into an abstract syntax tree (AST)
    tree = ast.parse(inspect.getsource(func))

    # Define a visitor class to traverse the AST and replace method calls with the underlying atomic operations
    class MethodCallVisitor(ast.NodeTransformer):
        def visit_Assign(self, node):
            # If the function being called is a method, replace the call with the method's body with parameters renamed
            if isinstance(node.func, ast.Attribute):
                method_name = node.func.attr
                for statement in func.__code__.co_consts:
                    if isinstance(statement, type((lambda: 0).__code__)):
                        if statement.co_name == method_name:
                            # Replace the method call with the body of the method with renamed parameters
                            method_ast = ast.parse(inspect.getsource(statement))
                            method_params = method_ast.body[0].args.args
                            method_args = node.args
                            for i in range(len(method_params)):
                                method_ast.body[0].args.args[i].arg = node.args[i].id
                            ast.copy_location(method_ast, node)
                            ast.fix_missing_locations(method_ast)
                            return method_ast.body[0]

            # Otherwise, visit the node's children
            return self.generic_visit(node)

    # Traverse the AST with the MethodCallVisitor
    transformer = MethodCallVisitor()
    new_tree = transformer.visit(tree)

    # Convert the AST back into Python code as a string
    return ast.unparse(new_tree)

In [22]:
action1 = lambda a: a + 2
action2 = lambda b: b + 3
action3 = lambda c: c - 4
action4 = lambda d: 2 * d
action5 = lambda e: e ** 2

def method1(a, b, c):
    action1(a)
    action2(b)
    action3(c)

def method2(d, e):
    action4(d)
    action5(e)

def method_complex(x, y, z, u, v):
    method1(x, y, z)
    method2(u, v)
    
# Add some additional calls on the same line to test the updated implementation
def method_complex2(x, y, z, u, v):
    method1(x, y, z); method2(u, v)
    method1(x, y, z); method2(u, v)

In [23]:
print(simplify(method_complex2))

def method_complex2(x, y, z, u, v):
    method1(x, y, z)
    method2(u, v)
    method1(x, y, z)
    method2(u, v)


In [24]:
class MethodCallVisitor(ast.NodeTransformer):
    def visit_Assign(self, node: ast.Assign):
        if not isinstance(node.value, ast.Call):
            return node
        new_node = []
        func: ast.Call = node.value
        funcDef: ast.FunctionDef = ast.parse(inspect.getsource(eval(func.func.id))).body[0]
        
        assign_left = [ast.Tuple([ast.Name(arg.arg) for arg in funcDef.args.args], ast.Store())]
        assign_right = ast.Tuple(func.args, ast.Load())
        
        new_node.append(ast.Assign(assign_left, assign_right, lineno=node.lineno))
        new_node.extend([i for i in funcDef.body if not isinstance(i, ast.Return)])
        
        returned: ast.Return = [i for i in funcDef.body if isinstance(i, ast.Return)][0]
        new_node.append(ast.Assign(node.targets, returned.value, lineno=node.lineno))
        
        return new_node

In [25]:
def print_ast(method):
    print(ast.dump(ast.parse(inspect.getsource(method)), indent=4, include_attributes=True))
    
def get_ast(method):
    return ast.parse(inspect.getsource(method))

In [26]:
def foo1(arg1, arg2, arg3):
    arg1 += arg2
    arg3 /= arg1
    arg2 += arg3
    arg1 /= arg2
    arg3 += arg1
    var1 = arg1 + arg2 + arg3
    var2 = arg1 - arg2 - arg3
    var3 = var1 - var2
    return var3

In [27]:
def any_function1(a1, b1, c1):
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    return a1, b1, c1

def any_function2(a2, b2, c2):
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    return d2, e2

def any_function3(d3, e3):
    f3 = d3 - e3
    return f3

def common_function(a, b, c):
    a, b, c = any_function1(a, b, c)
    d, e = any_function2(a, b, c)
    f = any_function3(d, e)
    return f

In [28]:
print_ast(any_function1)

Module(
    body=[
        FunctionDef(
            name='any_function1',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(
                        arg='a1',
                        lineno=1,
                        col_offset=18,
                        end_lineno=1,
                        end_col_offset=20),
                    arg(
                        arg='b1',
                        lineno=1,
                        col_offset=22,
                        end_lineno=1,
                        end_col_offset=24),
                    arg(
                        arg='c1',
                        lineno=1,
                        col_offset=26,
                        end_lineno=1,
                        end_col_offset=28)],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                AugAssign(
                    target=Name(
                        id='

In [29]:
mcv = MethodCallVisitor()
tree = get_ast(common_function)
tree = ast.fix_missing_locations(tree)

print(ast.unparse(mcv.visit(tree)))

def common_function(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f


In [30]:
def common_function_iteration1(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f

In [31]:
class AugAssignVisitor(ast.NodeTransformer):
    def visit_AugAssign(self, node: ast.AugAssign):
        return ast.Assign(
            targets=[node.target], 
            value=ast.BinOp(
                left=ast.Name(node.target.id, ast.Load()),
                op=node.op,
                right=node.value
            ),
            lineno=node.lineno
        )

In [32]:
aav = AugAssignVisitor()
new_tree = aav.visit(get_ast(common_function_iteration1))
new_tree = ast.fix_missing_locations(new_tree)
print(ast.unparse(new_tree))

def common_function_iteration1(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 = a1 + b1
    c1 = c1 / a1
    b1 = b1 + c1
    a1 = a1 / b1
    c1 = c1 + a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f


In [33]:
def common_function_iteration2(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 = a1 + b1
    c1 = c1 / a1
    b1 = b1 + c1
    a1 = a1 / b1
    c1 = c1 + a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f

In [34]:
class VariableCounter(ast.NodeVisitor):
    name_map = {}
    expr_num = 0
    
    def visit_arg(self, node: ast.arg):
        self.name_map[node.arg] = {
            'defines': [(-1, self.expr_num)],
            'usages': []
        }
        self.expr_num += 1

    def visit_Name(self, node: ast.Name):
        if node.id not in self.name_map:
            self.name_map[node.id] = {
                'defines': [],
                'usages': []
            }
        us_or_def = 'usages' if isinstance(node.ctx, ast.Load) else 'defines'
        self.name_map[node.id][us_or_def].append((node.lineno, self.expr_num))
        self.expr_num += 1

In [35]:
vc = VariableCounter()
vc.visit(get_ast(common_function_iteration2))

In [36]:
name_map = vc.name_map
name_map

{'a': {'defines': [(-1, 0), (8, 24)], 'usages': [(2, 6), (9, 33)]},
 'b': {'defines': [(-1, 1), (8, 25)], 'usages': [(2, 7), (9, 34)]},
 'c': {'defines': [(-1, 2), (8, 26)], 'usages': [(2, 8), (9, 35)]},
 'a1': {'defines': [(2, 3), (3, 9), (6, 18)],
  'usages': [(3, 10), (4, 14), (6, 19), (7, 23), (8, 27)]},
 'b1': {'defines': [(2, 4), (5, 15)],
  'usages': [(3, 11), (5, 16), (6, 20), (8, 28)]},
 'c1': {'defines': [(2, 5), (4, 12), (7, 21)],
  'usages': [(4, 13), (5, 17), (7, 22), (8, 29)]},
 'a2': {'defines': [(9, 30)], 'usages': [(10, 37), (11, 41)]},
 'b2': {'defines': [(9, 31)], 'usages': [(10, 38), (11, 42)]},
 'c2': {'defines': [(9, 32)], 'usages': [(10, 39), (11, 43)]},
 'd2': {'defines': [(10, 36)], 'usages': [(12, 46)]},
 'e2': {'defines': [(11, 40)], 'usages': [(12, 47)]},
 'd': {'defines': [(12, 44)], 'usages': [(13, 50)]},
 'e': {'defines': [(12, 45)], 'usages': [(13, 51)]},
 'd3': {'defines': [(13, 48)], 'usages': [(14, 53)]},
 'e3': {'defines': [(13, 49)], 'usages': [(14,

In [37]:
new_name_map = {}
table = {}
for name in name_map:
    defines = name_map[name]['defines']
    usages = name_map[name]['usages']
    
    for define, i in zip(defines, range(len(defines))):
        new_name_map[f"{name}___definition___{i}"] = {'define': define, 'usages': []}
        
    for usage in usages:
        for i in range(len(defines)):
            if (i == len(defines) - 1) or ((defines[i][0] < usage[0]) and (defines[i+1][0] >= usage[0])):
                new_name_map[f"{name}___definition___{i}"]['usages'].append(usage)
                break

In [38]:
new_name_map

{'a___definition___0': {'define': (-1, 0), 'usages': [(2, 6)]},
 'a___definition___1': {'define': (8, 24), 'usages': [(9, 33)]},
 'b___definition___0': {'define': (-1, 1), 'usages': [(2, 7)]},
 'b___definition___1': {'define': (8, 25), 'usages': [(9, 34)]},
 'c___definition___0': {'define': (-1, 2), 'usages': [(2, 8)]},
 'c___definition___1': {'define': (8, 26), 'usages': [(9, 35)]},
 'a1___definition___0': {'define': (2, 3), 'usages': [(3, 10)]},
 'a1___definition___1': {'define': (3, 9), 'usages': [(4, 14), (6, 19)]},
 'a1___definition___2': {'define': (6, 18), 'usages': [(7, 23), (8, 27)]},
 'b1___definition___0': {'define': (2, 4), 'usages': [(3, 11), (5, 16)]},
 'b1___definition___1': {'define': (5, 15), 'usages': [(6, 20), (8, 28)]},
 'c1___definition___0': {'define': (2, 5), 'usages': [(4, 13)]},
 'c1___definition___1': {'define': (4, 12), 'usages': [(5, 17), (7, 22)]},
 'c1___definition___2': {'define': (7, 21), 'usages': [(8, 29)]},
 'a2___definition___0': {'define': (9, 30), 

In [39]:
import pandas as pd
res = []
for name in new_name_map:
    res.append({
        'name': name,
        'define': new_name_map[name]['define'],
        'last_usage': new_name_map[name]['usages'][-1],
        'usages': new_name_map[name]['usages']
    })
res = pd.DataFrame(res).sort_values("last_usage").reset_index(drop=True)
res

,name,define,last_usage,usages
0,a___definition___0,"(-1, 0)","(2, 6)","[(2, 6)]"
1,b___definition___0,"(-1, 1)","(2, 7)","[(2, 7)]"
2,c___definition___0,"(-1, 2)","(2, 8)","[(2, 8)]"
3,a1___definition___0,"(2, 3)","(3, 10)","[(3, 10)]"
4,c1___definition___0,"(2, 5)","(4, 13)","[(4, 13)]"
5,b1___definition___0,"(2, 4)","(5, 16)","[(3, 11), (5, 16)]"
6,a1___definition___1,"(3, 9)","(6, 19)","[(4, 14), (6, 19)]"
7,c1___definition___1,"(4, 12)","(7, 22)","[(5, 17), (7, 22)]"
8,a1___definition___2,"(6, 18)","(8, 27)","[(7, 23), (8, 27)]"
9,b1___definition___1,"(5, 15)","(8, 28)","[(6, 20), (8, 28)]"


In [40]:
sorted_by_usage = res.sort_values("last_usage")
sorted_by_define = res.sort_values("define")

nexts = []
for i in range(len(sorted_by_usage)):
    for j in range(i+1, len(sorted_by_define)):
        if j in nexts:
            continue
        if sorted_by_usage.iloc[i].last_usage[0] <= sorted_by_define.iloc[j].define[0]:
            nexts.append(j)
            break
    if len(nexts) != i + 1:
        nexts.append(None)

In [41]:
for next_, i in zip(nexts, range(len(nexts))):
    if next_ is None:
        continue
    sorted_by_usage.iloc[next_, 0] = sorted_by_usage.iloc[i, 0]
    
sorted_by_usage

,name,define,last_usage,usages
0,a___definition___0,"(-1, 0)","(2, 6)","[(2, 6)]"
1,b___definition___0,"(-1, 1)","(2, 7)","[(2, 7)]"
2,c___definition___0,"(-1, 2)","(2, 8)","[(2, 8)]"
3,a___definition___0,"(2, 3)","(3, 10)","[(3, 10)]"
4,b___definition___0,"(2, 5)","(4, 13)","[(4, 13)]"
5,c___definition___0,"(2, 4)","(5, 16)","[(3, 11), (5, 16)]"
6,a___definition___0,"(3, 9)","(6, 19)","[(4, 14), (6, 19)]"
7,b___definition___0,"(4, 12)","(7, 22)","[(5, 17), (7, 22)]"
8,c___definition___0,"(6, 18)","(8, 27)","[(7, 23), (8, 27)]"
9,a___definition___0,"(5, 15)","(8, 28)","[(6, 20), (8, 28)]"


In [42]:
from sklearn.preprocessing import LabelEncoder

In [43]:
le = LabelEncoder()
sorted_by_usage.name = [f"var_{i}" for i in le.fit_transform(sorted_by_usage.name)]
sorted_by_usage

,name,define,last_usage,usages
0,var_0,"(-1, 0)","(2, 6)","[(2, 6)]"
1,var_1,"(-1, 1)","(2, 7)","[(2, 7)]"
2,var_2,"(-1, 2)","(2, 8)","[(2, 8)]"
3,var_0,"(2, 3)","(3, 10)","[(3, 10)]"
4,var_1,"(2, 5)","(4, 13)","[(4, 13)]"
5,var_2,"(2, 4)","(5, 16)","[(3, 11), (5, 16)]"
6,var_0,"(3, 9)","(6, 19)","[(4, 14), (6, 19)]"
7,var_1,"(4, 12)","(7, 22)","[(5, 17), (7, 22)]"
8,var_2,"(6, 18)","(8, 27)","[(7, 23), (8, 27)]"
9,var_0,"(5, 15)","(8, 28)","[(6, 20), (8, 28)]"


In [44]:
sorted_by_usage.define = sorted_by_usage.define.apply(lambda elem: elem[1])
sorted_by_usage.usages = sorted_by_usage.usages.apply(lambda elem: [j[1] for j in elem])
sorted_by_usage

,name,define,last_usage,usages
0,var_0,0,"(2, 6)",[6]
1,var_1,1,"(2, 7)",[7]
2,var_2,2,"(2, 8)",[8]
3,var_0,3,"(3, 10)",[10]
4,var_1,5,"(4, 13)",[13]
5,var_2,4,"(5, 16)","[11, 16]"
6,var_0,9,"(6, 19)","[14, 19]"
7,var_1,12,"(7, 22)","[17, 22]"
8,var_2,18,"(8, 27)","[23, 27]"
9,var_0,15,"(8, 28)","[20, 28]"


In [45]:
sorted_by_usage.usages = sorted_by_usage.apply(lambda row: row.usages + [row.define], axis=1)
sorted_by_usage

,name,define,last_usage,usages
0,var_0,0,"(2, 6)","[6, 0]"
1,var_1,1,"(2, 7)","[7, 1]"
2,var_2,2,"(2, 8)","[8, 2]"
3,var_0,3,"(3, 10)","[10, 3]"
4,var_1,5,"(4, 13)","[13, 5]"
5,var_2,4,"(5, 16)","[11, 16, 4]"
6,var_0,9,"(6, 19)","[14, 19, 9]"
7,var_1,12,"(7, 22)","[17, 22, 12]"
8,var_2,18,"(8, 27)","[23, 27, 18]"
9,var_0,15,"(8, 28)","[20, 28, 15]"


In [46]:
var_list = [''] * vc.expr_num
for row in sorted_by_usage.iloc:
    for i in row.usages:
        var_list[i] = row["name"]
var_list

['var_0',
 'var_1',
 'var_2',
 'var_0',
 'var_2',
 'var_1',
 'var_0',
 'var_1',
 'var_2',
 'var_0',
 'var_0',
 'var_2',
 'var_1',
 'var_1',
 'var_0',
 'var_0',
 'var_2',
 'var_1',
 'var_2',
 'var_0',
 'var_0',
 'var_1',
 'var_1',
 'var_2',
 'var_2',
 'var_0',
 'var_1',
 'var_2',
 'var_0',
 'var_1',
 'var_2',
 'var_0',
 'var_1',
 'var_2',
 'var_0',
 'var_1',
 'var_3',
 'var_2',
 'var_0',
 'var_1',
 'var_2',
 'var_2',
 'var_0',
 'var_1',
 'var_0',
 'var_1',
 'var_3',
 'var_2',
 'var_3',
 'var_2',
 'var_0',
 'var_1',
 'var_0',
 'var_3',
 'var_2',
 'var_1',
 'var_0',
 'var_1']

In [47]:
class VariableTransformer(ast.NodeTransformer):
    expr_num = 0
    
    def visit_arg(self, node: ast.arg):
        node.arg = var_list[self.expr_num]
        self.expr_num += 1
        return node

    def visit_Name(self, node: ast.Name):
        node.id = var_list[self.expr_num]
        self.expr_num += 1
        return node

In [48]:
vt = VariableTransformer()

print(ast.unparse(vt.visit(get_ast(common_function_iteration2))))

def common_function_iteration2(var_0, var_1, var_2):
    (var_0, var_2, var_1) = (var_0, var_1, var_2)
    var_0 = var_0 + var_2
    var_1 = var_1 / var_0
    var_0 = var_2 + var_1
    var_2 = var_0 / var_0
    var_1 = var_1 + var_2
    (var_2, var_0, var_1) = (var_2, var_0, var_1)
    (var_2, var_0, var_1) = (var_2, var_0, var_1)
    var_3 = var_2 + var_0 + var_1
    var_2 = var_2 - var_0 - var_1
    (var_0, var_1) = (var_3, var_2)
    (var_3, var_2) = (var_0, var_1)
    var_0 = var_3 - var_2
    var_1 = var_0
    return var_1


In [11]:
import ast
import pandas as pd
import inspect
from sklearn.preprocessing import LabelEncoder

class AugAssignVisitor(ast.NodeTransformer):
    def visit_AugAssign(self, node: ast.AugAssign):
        return ast.Assign(
            targets=[node.target], 
            value=ast.BinOp(
                left=ast.Name(node.target.id, ast.Load()),
                op=node.op,
                right=node.value
            ),
            lineno=node.lineno
        )
    
class VariableCounter(ast.NodeVisitor):
    name_map = {}
    expr_num = 0
    
    def visit_arg(self, node: ast.arg):
        self.name_map[node.arg] = {
            'defines': [(-1, self.expr_num)],
            'usages': []
        }
        self.expr_num += 1

    def visit_Name(self, node: ast.Name):
        if node.id not in self.name_map:
            self.name_map[node.id] = {
                'defines': [],
                'usages': []
            }
        us_or_def = 'usages' if isinstance(node.ctx, ast.Load) else 'defines'
        self.name_map[node.id][us_or_def].append((node.lineno, self.expr_num))
        self.expr_num += 1
        
        
class VariableTransformer(ast.NodeTransformer):
    expr_num = 0
    def __init__(self, var_list):
        super().__init__()
        self.var_list = var_list
        
    def visit_arg(self, node: ast.arg):
        node.arg = self.var_list[self.expr_num]
        self.expr_num += 1
        return node

    def visit_Name(self, node: ast.Name):
        node.id = self.var_list[self.expr_num]
        self.expr_num += 1
        return node
    
def print_ast(method):
    print(ast.dump(ast.parse(inspect.getsource(method)), indent=4, include_attributes=True))
    
def get_ast(method):
    return ast.parse(inspect.getsource(method))
        
def reform_name_map(name_map):
    new_name_map = {}
    for name in name_map:
        defines = name_map[name]['defines']
        usages = name_map[name]['usages']

        for define, i in zip(defines, range(len(defines))):
            new_name_map[f"{name}___definition___{i}"] = {'define': define, 'usages': []}

        for usage in usages:
            for i in range(len(defines)):
                if (i == len(defines) - 1) or ((defines[i][0] < usage[0]) and (defines[i+1][0] >= usage[0])):
                    new_name_map[f"{name}___definition___{i}"]['usages'].append(usage)
                    break
    return new_name_map
                    
                    
def name_map_to_pd(name_map):
    res = []
    for name in name_map:
        res.append({
            'name': name,
            'define': name_map[name]['define'],
            'last_usage': None if len(name_map[name]['usages']) == 0 else name_map[name]['usages'][-1],
            'usages': name_map[name]['usages']
        })
    return pd.DataFrame(res)


def calculate_nexts(pd_name_map):
    sorted_by_usage = pd_name_map.sort_values("last_usage").reset_index(drop=True)
    sorted_by_define = pd_name_map.sort_values("define").reset_index(drop=True)

    nexts = []
    for i in range(len(sorted_by_usage)):
        for j in range(i+1, len(sorted_by_define)):
            if j in nexts:
                continue
            if sorted_by_usage.iloc[i].last_usage[0] <= sorted_by_define.iloc[j].define[0]:
                nexts.append(j)
                break
        if len(nexts) != i + 1:
            nexts.append(None)
            
    return sorted_by_usage, nexts


def apply_nexts(sorted_by_usage, nexts):
    for next_, i in zip(nexts, range(len(nexts))):
        if next_ is None:
            continue
        sorted_by_usage.iloc[next_, 0] = sorted_by_usage.iloc[i, 0]
    return sorted_by_usage


def anonimize_vars(sorted_by_usage):
    le = LabelEncoder()
    sorted_by_usage.name = [f"var_{i}" for i in le.fit_transform(sorted_by_usage.name)]
    return sorted_by_usage

    
def get_var_list(sorted_by_usage, expr_num):
    sorted_by_usage.define = sorted_by_usage.define.apply(lambda elem: elem[1])
    sorted_by_usage.usages = sorted_by_usage.usages.apply(lambda elem: [j[1] for j in elem])
    
    sorted_by_usage.usages = sorted_by_usage.apply(lambda row: row.usages + [row.define], axis=1)
    
    var_list = [''] * expr_num
    for row in sorted_by_usage.iloc:
        for i in row.usages:
            var_list[i] = row["name"]
            
    return var_list
    
    
def clear_variables(ast_tree):
    aav = AugAssignVisitor()
    ast_tree = ast.fix_missing_locations(aav.visit(ast_tree))
    
    vc = VariableCounter()
    vc.visit(ast_tree)
    
    name_map = reform_name_map(vc.name_map)
    
    res = name_map_to_pd(name_map)
    sorted_by_usage, nexts = calculate_nexts(res)
    sorted_by_usage = apply_nexts(sorted_by_usage, nexts)
    sorted_by_usage = anonimize_vars(sorted_by_usage)
    
    var_list = get_var_list(sorted_by_usage, vc.expr_num)
    
    vt = VariableTransformer(var_list)
    
    return vt.visit(ast_tree)

In [12]:
print(ast.unparse(clear_variables(get_ast(common_function_iteration1))))

def common_function_iteration1(var_0, var_1, var_2):
    (var_0, var_2, var_1) = (var_0, var_1, var_2)
    var_0 = var_0 + var_2
    var_1 = var_1 / var_0
    var_0 = var_2 + var_1
    var_2 = var_0 / var_0
    var_1 = var_1 + var_2
    (var_2, var_0, var_1) = (var_2, var_0, var_1)
    (var_2, var_0, var_1) = (var_2, var_0, var_1)
    var_3 = var_2 + var_0 + var_1
    var_2 = var_2 - var_0 - var_1
    (var_0, var_1) = (var_3, var_2)
    (var_3, var_2) = (var_0, var_1)
    var_0 = var_3 - var_2
    var_1 = var_0
    return var_1


In [57]:
def common_function_iteration2(var_4, var_1, var_2):
    (var_4, var_0, var_1) = (var_4, var_1, var_2)
    var_4 = var_4 + var_0
    var_1 = var_1 / var_4
    var_4 = var_0 + var_1
    var_2 = var_4 / var_4
    var_1 = var_1 + var_2
    (var_2, var_4, var_1) = (var_2, var_4, var_1)
    (var_2, var_4, var_1) = (var_2, var_4, var_1)
    var_7 = var_2 + var_4 + var_1
    var_2 = var_2 - var_4 - var_1
    (var_4, var_1) = (var_7, var_2)
    (var_7, var_2) = (var_4, var_1)
    var_4 = var_7 - var_2
    var_1 = var_4
    return var_1

In [9]:
def common_function_iteration1(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 = a1 + b1
    c1 = c1 / a1
    b1 = b1 + c1
    a1 = a1 / b1
    c1 = c1 + a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f

In [8]:
print(inspect.getsource(common_function_iteration1))

def common_function_iteration1(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 = a1 + b1
    c1 = c1 / a1
    b1 = b1 + c1
    a1 = a1 / b1
    c1 = c1 + a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f

